# References

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
#sys.path.append('../fastai/old/') #fastai version 0.7
sys.path.append('../fastai/') #fastai version 1

from fastai import *

import pdb
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
import png
from tqdm import tqdm_notebook as tqdm

In [2]:
torch.cuda.set_device(1)
torch.backends.cudnn.benchmark=True

# Global Variables

In [3]:
DP = Path('../datasets/human-protein-atlas-image-classification/')

TRAIN_PNGS = DP/'train_pngs'

TEST_ONE = 'test1_pngs'

SUBMISSIONS = DP/'submissions'
SUBMISSIONS.mkdir(exist_ok=True)